In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from boxes import Box, BoxCollection

# Offline

In [ ]:
boxes = [
    Box([6.25,    4], [7.25, 5.75]),
    Box([ 5.5,  2.5], [ 7.5, 4.75]),
    Box([   1,    4], [   6,    5]),
    Box([ .25,  1.5], [   7,    3]),
    Box([2.25,  .75], [   3,  2.5]),
    Box([  .5, -.25], [ 1.5,  4.5]),
    Box([   2,    0], [6.25,    1]),
    Box([4.75, -.25], [   6, 3.75]),
    Box([   0,  5.2], [   7,    6]),
]
B = BoxCollection(boxes)

In [ ]:
u = np.array([b.u for b in boxes])
l = np.array([b.l for b in boxes])
frame = .1
lims = np.vstack([
    np.min(l, axis=0) - frame,
    np.max(u, axis=0) + frame
])
figsize = (lims[1] - lims[0]) / 2.5

def initialize_figure():
    plt.figure(figsize=figsize)
    plt.gca().set_aspect('equal')
    plt.axis('off')
    plt.xlim(lims[:, 0])
    plt.ylim(lims[:, 1])

In [ ]:
initialize_figure()
B.plot2d(alpha=.75)
# plt.savefig('boxes.pdf', bbox_inches='tight')

In [ ]:
G_center = B.line_graph('centers')
intersections = [G_center.nodes[v]['box'] for v in G_center.nodes]
I = BoxCollection(intersections)

In [ ]:
initialize_figure()
B.plot2d(alpha=.25)
I.plot2d(fc='lightcoral', alpha=.75)
# plt.savefig('intersections.pdf', bbox_inches='tight')

In [ ]:
initialize_figure()
B.plot2d(alpha=.25)
G_center.plot(c='b')
I.plot2d(fc='lightcoral', alpha=.75)
# plt.savefig('line_graph_centers.pdf', bbox_inches='tight')

In [ ]:
G_opt = B.line_graph('optimize')
initialize_figure()
B.plot2d(alpha=.25)
I.plot2d(fc='lightcoral', alpha=.75)
G_opt.plot(c='b')
# plt.savefig('line_graph_optimized.pdf', bbox_inches='tight')

# Online

In [ ]:
start = np.array([1, .25])
goal = np.array([.5, 5.6])

In [ ]:
G_opt.add_node('s')
G_opt.nodes['s']['point'] = start

for k in B.contain(start):
    for l in B.inters[k]:
        v = G_opt.node(k, l)
        e = ('s', v)
        G_opt.add_edge(*e)
        pv = G_opt.nodes[v]['point']
        G_opt.edges[e]['weight'] = np.linalg.norm(start - pv)
        
G_opt.add_node('t')
G_opt.nodes['t']['point'] = goal

for k in B.contain(goal):
    for l in B.inters[k]:
        v = G_opt.node(k, l)
        e = (v, 't')
        G_opt.add_edge(*e)
        pv = G_opt.nodes[v]['point']
        G_opt.edges[e]['weight'] = np.linalg.norm(goal - pv)

In [ ]:
initialize_figure()
B.plot2d(alpha=.25)
I.plot2d(fc='lightcoral', alpha=.75)
G_opt.plot(c='b')
d = np.array([.2, 0])
plt.scatter(*start, marker='o', fc='k', ec='k', zorder=5)
plt.scatter(*goal, marker='o', fc='k', ec='k', zorder=5)
# plt.savefig('boundary_conditions.pdf', bbox_inches='tight')

In [ ]:
import networkx as nx
path = nx.shortest_path(G_opt, source='s', target='t', weight='weight')
traj = np.array([G_opt.nodes[v]['point'] for v in path])

In [ ]:
initialize_figure()
B.plot2d(alpha=.25)
path_nodes = [list(G_opt.nodes).index(v) for v in path]
I.plot2d(fc='lightcoral', alpha=.75, subset=path_nodes)
plt.plot(*traj.T, c='b', marker='o', markerfacecolor='w', markeredgecolor='k')
plt.scatter(*start, marker='o', fc='k', ec='k', zorder=5)
plt.scatter(*goal, marker='o', fc='k', ec='k', zorder=5)
# plt.savefig('spp.pdf', bbox_inches='tight')

In [ ]:
from polygonal import build_min_distance

G_opt = B.line_graph('optimize')
discrete_planner, runtime = G_opt.shortest_path(goal)
box_seq, length, runtime = discrete_planner(start)

max_boxes = len(box_seq)
continuous_planner = build_min_distance(B, max_boxes, solver='ECOS')
traj, length, solver_time, runtime = continuous_planner(box_seq, start, goal)

initialize_figure()
B.plot2d(alpha=.25)
B.plot2d(fc='moccasin', alpha=.75, subset=box_seq)
plt.plot(*traj.T, c='b', marker='o', markerfacecolor='w', markeredgecolor='k')
plt.scatter(*start, marker='o', fc='k', ec='k', zorder=5)
plt.scatter(*goal, marker='o', fc='k', ec='k', zorder=5)
# plt.savefig('first_refine.pdf', bbox_inches='tight')

In [ ]:
box_seq = [5, 3, 7, 1, 0, 8]

max_boxes = len(box_seq)
continuous_planner = build_min_distance(B, max_boxes, solver='ECOS')
traj, length, solver_time, runtime = continuous_planner(box_seq, start, goal)

initialize_figure()
B.plot2d(alpha=.25)
B.plot2d(fc='moccasin', alpha=.75, subset=[i for i in box_seq if i != 7])
B.plot2d(fc='lightgreen', alpha=.75, subset=[7])
plt.plot(*traj.T, c='b', marker='o', markerfacecolor='w', markeredgecolor='k')
plt.scatter(*start, marker='o', fc='k', ec='k', zorder=5)
plt.scatter(*goal, marker='o', fc='k', ec='k', zorder=5)
# plt.savefig('second_refine.pdf', bbox_inches='tight')

In [ ]:
initialize_figure()
B.plot2d(alpha=.25)
B.plot2d(fc='moccasin', alpha=.75, subset=box_seq)
plt.plot(*traj.T, c='b', marker='o', markerfacecolor='w', markeredgecolor='k')
plt.scatter(*start, marker='o', fc='k', ec='k', zorder=5)
plt.scatter(*goal, marker='o', fc='k', ec='k', zorder=5)
# plt.savefig('polygonal.pdf', bbox_inches='tight')

In [ ]:
import cvxpy as cp
from bezier import optimize_bezier, optimize_bezier_with_retiming

l = np.array([B.boxes[i].l for i in box_seq])
u = np.array([B.boxes[i].u for i in box_seq])

lam = {3: 1}
initial = {0: start}
final = {0: goal}

T = 10
h0 = np.linalg.norm(traj[1:] - traj[:-1], axis=1)
h0 *= T / sum(h0)

In [ ]:
bez0, cost0, runtime0 = optimize_bezier(l, u, h0, lam, initial, final)[:-1]
cost0, runtime0

In [ ]:
bez, cost, h, runtime = optimize_bezier_with_retiming(l, u, h0, lam,
                                                                        initial, final,
                                                                        verbose=True)

In [ ]:
plt.rcParams.update({""
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
})

In [ ]:
dd = .1
def position(box, pos):
    c = (box.l + box.u) / 2
    if pos == 'center':
        return c, 'center', 'center'
    if pos == 'left':
        c[0] = box.l[0] + dd
        return c, 'left', 'center'
    if pos == 'right':
        c[0] = box.u[0]
        return c, 'right', 'center'
    if pos == 'bottom':
        c[1] = box.l[1]
        return c, 'center', 'bottom'
    if pos == 'top':
        c[1] = box.u[1] - dd
        return c, 'center', 'top'

In [ ]:
idx = [0, 1, -1]
trajs = [bez0, bez0, bez]

positions = [
    ['center', 'top', 'center', 'center', 'right', 'top'],
    ['center', 'bottom', 'center', 'right', 'right', 'bottom'],
    ['center', 'bottom', 'center', 'right', 'right', 'bottom']
]
for k, i in enumerate(idx):
    
    initialize_figure()
    B.plot2d(alpha=.25)
    B.plot2d(fc='moccasin', alpha=.75, subset=box_seq)
    trajs[i].plot(c='b', zorder=5)
    trajs[i].plot_control_polytopes(fc='b', alpha=.2)
    for b in trajs[i].beziers:
        plt.scatter(*b.points[0], c='b', marker='.')
    for jj, j in enumerate(box_seq):
        point = B.boxes[j].c
        point, ha, va = position(B.boxes[j], positions[i][jj])
#         plt.text(*point, f'${round(hs[k, jj] * 100 / T)}\%$',
#                  c='k', zorder=5, ha=ha, va=va)
        
    plt.scatter(*start, marker='o', fc='k', ec='k', zorder=5)
    plt.scatter(*goal, marker='o', fc='k', ec='k', zorder=5)
#     plt.savefig(f'bezier{k}.pdf', bbox_inches='tight')